# Marketing Campaign Optimization

## Objective and Prerequisites

This Jupyter Notebook  describes a marketing campaign optimization problem that is common in 
the banking and financial services industry. 
The problem is formulated using the Gurobi Python API and solved using the Gurobi Optimizer. 
We assume that key parameters of the mathematical optimization model of the marketing campaign problem are estimated using machine learning predictive response models. 
The marketing campaign optimization problem entails determining which products to offer to each customer 
in order 
to maximize total expected profit while satisfying various business constraints.

This modeling example is at the beginner level, where we assume that you know Python and that you have some knowledge about building mathematical optimization models. The reader should also consult the  [documentation](https://www.gurobi.com/resources/?category-filter=documentation)
of the Gurobi Python API.

**Note:** You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=CommercialDataScience) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=AcademicDataScience) as an *academic user*.

## Motivation

The main goal of marketing in the banking and financial services industry is to offer "the right product to the right customer at the right time". However, 
actually being able to achieve this goal is a complicated and challenging undertaking. What makes this particularly difficult is that companies have multiple products and operate under a complex set of business constraints. Choosing which products to offer to which customers in order to maximize the marketing return on investment and satisfy the business constraints is enormously complex.

Consider a major bank  that has made a deliberate effort to become a customer-focused institution, as opposed to a vertical product driven company. The goal of the bank is "to be the best at helping customers become financially better off by providing relevant solutions to their unique needs". A direct consequence of this goal is that marketing campaigns are multiple product campaigns as opposed to single product campaigns. 
This transforms the data science and campaign targeting process from a fairly simple application of individual response models into a significantly more complex process that invloves  choosing which product to offer to which customer and through which channel.

The marketing team of the bank are used to applying business rules to target customers directly. 
For example, 
they target 
customers solely on their product gaps or on marketers' business intuition. 
The bank's
marketers have also applied RFM type analysis where general recency, frequency, and monetary measurements as well as product gaps are used to target customers for specific offers. 

The marketing team's current approach, which 
is widely used, 
relies on predictive response models to target customers for offers. These models estimate the probability that a customer will respond to a specific offer and can significantly increase the response rate to a product offering. However, simply knowing a customer's probability of responding to a particular offer is not enough when a company has several products to promote and other business constraints to consider in its marketing planning.

Generally speaking, marketing teams also face the problem of knowing which product to offer to a customer, not just which customer to offer a product. In practice, many ad hoc rules are used:

* prioritization rules based on response rates or estimated expected profitability measures

* business rules to prioritize products that can be marketed

* product response models to select customers for a particular campaign. 

One approach that is easily implemented but may not produce optimal customer contact plans relies on a measure of expected offer profitability to choose which products to offer customers. However, a shortcoming of this approach is its inability to effectively handle complex constraints on the customer contact plan.

To address this marketing campaign optimization problem, M. D. Cohen [1] proposed a MIP approach with data from Scotiabank. The marketing campaign optimization problem considered eleven unique offers: five investment, three lending, and three day-to-day banking offers. The investment offers included Guaranteed Investment Certificates (GICs), mutual funds, Registered Education Savings Program (RESP) and two unique discount brokerage offers. The lending offers included a mortgage and two credit card offers. 
The day-to-day banking offers included one of two Scotia online banking service offers and a deposit account acquisition. The term campaign is used here to imply one large pro-active customer contact campaign that is comprised of eleven distinct offers; it can be thought of as eleven single product campaigns that are being offered at generally the same time to a non-overlapping set of customers. Approximately 2.5 million customers were included in the potential target market for the campaign.


In this Jupyter Notebook, we will use this MIP approach to address the bank’s marketing campaign optimization problem. It should be noted that this approach could be used by virtually any company across various industries to optimize their marketing campaigns, while taking into account their business constraints. 


## Problem Description
The bank's marketing team needs to determine what products to offer to each customer in a way that maximizes the marketing campaign return on investment while considering the following constraints: 

* limits on funding available for the campaign.
* restrictions on the minimum number of product offers that can be made in a campaign.
* campaign return-on-investment hurdle rates that must be met.

## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP approach for this marketing campaign optimization problem.

The MIP solution approach proposed by Cohen [1] is an improvement over the 
traditional myopic
approach of picking the customers that have the largest expected value for a particular product because it
produces a globally optimal solution 
from the viewpoint of the bank and allows for the effective implementation of business constraints across customers and business units. The approach accounts for limited resources and other business constraints. 

We assume that the estimates for customer/offer expected incremental profit, costs, and business constraints serve as inputs to the marketing campaign optimization approach. The optimization phase is independent of the construction of these inputs. 

The MIP approach
involves 
a tactical and an operational problem. 
For the tactical problem, we aggregate customers based on the individual expected profit parameters.  The estimated individual expected profit can be determined with data science techniques such as predictive response models. The key idea is to cluster the estimated individual expected profits and then consider the cluster centroids as representative of the data for all the individual customers within a single cluster. This aggregation enables the problem to be formulated as a linear programming problem so that rather than assigning offers to individual customers, the model identifies proportions within each cluster for each product offer that maximizes the marketing campaign return on investment while considering the business constraints. Typically, the number of customers in a cluster will be in the hundreds of thousands, which is the main decision variable of the tactical problem, consequently these variables can be considered as a continuous; therefore, the linear programming approach is justified.

The operational problem can be formulated as a MIP model, where the estimated individual expected profits and the output of the tactical model can be used as inputs to assign *products offers* to individual customers of each cluster in such a way that the  total marketing campaign return on investment is maximized.

## Tactical Model Formulation

### Sets and Indices
$k \in K$: Index and set of clusters.

$j \in J$: Index and set of products.

### Parameters
$\pi_{k,j}$: Expected profit to the bank from the offer of product $j \in J$ to an average customer of cluster $k \in K$.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.
  
$N_{k}$: Number of customers in cluster $k \in K$.

$Q_{j}$: Minimum number of offers of product $j \in J$. 

$R$: Corporate hurdle rate. This hurdle rate is used for the ROI calculation of the marketing campaign.

$B$: Marketing campaign budget.

$M$: Big M penalty. This penalty is associated with corrections on the budget that are necessary to satisfy other business constraints.
 

### Decision Variables
$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

### Objective Function
- **Total profit**. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}

### Constraints

- **Number of offers**. Maximum number of offers of products for each cluster is limited by the number of customers in the cluster.

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

- **Budget**. The marketing campaign budget constraint enforces that the total cost of the campaign should be less than the budget campaign. There is the possibility of increasing the budget to ensure the feasibility of the model, the minimum number of offers for all the product may require this increase in the budget.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}

- **Offers limit**. Minimum number of offers of each product.

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

- **ROI**. The minimum ROI constraint ensures that the ratio of total profits over cost is at least one plus the corporate hurdle rate.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}


## Operational Model Formulation

Once the optimal values $y_{k,j}$, for all $j \in J$ and $k \in K$, of the Tactical model have been found, we should determine which individual customers in cluster $k$ should get an offer of a product. Suppose that for a given cluster $k \in K$, the allocation of offers of product $j_1$ and $j_2$ are positive, i.e. $y_{k,j_1} > 0$ and $y_{k,j_2} > 0$. Then, $y_{k,j_1}$ and $y_{k,j_2}$ of customers in cluster $k$ must be offered product $j_1$ and $j_2$, respectively. The optimal way to do that is to solve an assignment problem using the estimated expected profit for the individual customers and not the one for clusters.

We now provide a formulation of the operational problem.

### Sets and Indices
$i \in I^{k}$: Index and set of customers in cluster $k \in K$.

$j \in J^{k}$: Index and subset of products offered to customers in cluster $k \in K$ , where $J^{k} = \{ j \in J: y_{k,j} > 0 \}$ .

### Parameters

$r_{k,i,j}$: Expected individual profit of customer $i \in I^{k}$ from  offer of product $j \in J^{k}$. 

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: Number of customers in cluster k that will get an offer of product $j \in J^{k}$.

### Decision Variables
$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.



### Objective Function
- **Total profit**. Maximize total individual profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}


### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}


- **Offers limit**. At most one product may be offered to a customer of a cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **Binary constraints**. Either a product offer is given to a customer of cluster k or not.

\begin{equation}
x_{k,i,j} \in \{0,1 \} \quad \forall i \in I^{k},  j \in J^{k}, k \in K
\tag{3}
\end{equation}


## Problem Instance

We consider two products, ten customers, and two clusters of customers. The corporate hurdle-rate is twenty percent.

### Tactical problem data

The following table defines the expected profit of an average customer in each cluster when offered a product.

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$2,000$ | $\$1,000$ |
| cluster 2 | $\$3,000$ | $\$2,000$ |

The expected cost of offering a product to an average customer in a cluster is determined by the following table.

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$200$ | $\$100$ |
| cluster 2 | $\$300$ | $\$200$ |

The budget available for the marketing campaign is $\$200$.

The number of customers in each cluster is given by the following table.

| <i></i> | Num. Customers | 
| --- | --- |
| cluster 1 | 5 |
| cluster 2 | 5 | 

The minimum number of offers of each product is provided in the following table,

| <i></i> | Min Offers | 
| --- | --- |
| product 1 | 2 |
| product 2 | 2 | 

### Operational problem data

The following table shows the expected profit of each customer in each cluster when offered a product.

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1, customer 1 | $\$2,050$ | $\$1,050$ |
| cluster 1, customer 2 | $\$1,950$ | $\$950$ |
| cluster 1, customer 3 | $\$2,000$ | $\$1,000$ |
| cluster 1, customer 4 | $\$2,100$ | $\$1,100$ |
| cluster 1, customer 5 | $\$1,900$ | $\$900$ |
| cluster 2, customer 6 | $\$3,000$ | $\$2,000$ |
| cluster 2, customer 7 | $\$2,900$ | $\$1,900$ |
| cluster 2, customer 8 | $\$3,050$ | $\$2,050$ |
| cluster 2, customer 9 | $\$3,100$ | $\$2,100$ |
| cluster 2, customer 10 | $\$2,950$ | $\$1,950$ |

The following table shows the cost of offering a product to a customer in a cluster.

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1, customer 1 | $\$205$ | $\$105$ |
| cluster 1, customer 2 | $\$195$ | $\$95$ |
| cluster 1, customer 3 | $\$200$ | $\$100$ |
| cluster 1, customer 4 | $\$210$ | $\$110$ |
| cluster 1, customer 5 | $\$190$ | $\$90$ |
| cluster 2, customer 6 | $\$300$ | $\$200$ |
| cluster 2, customer 7 | $\$290$ | $\$190$ |
| cluster 2, customer 8 | $\$305$ | $\$205$ |
| cluster 2, customer 9 | $\$310$ | $\$210$ |
| cluster 2, customer 10 | $\$295$ | $\$195$ |

## Python Implementation

We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.

In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.0.0 and Python 3.7.0

### SETS

products = ['p1', 'p2']
clusters = ['k1', 'k2']

### Expected profit

The following tables shows the expected profit of a customer in each cluster when offered a product.

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$2,000$ | $\$1,000$ |
| cluster 2 | $\$3,000$ | $\$2,000$ |

In [2]:
### Parameters

# Expected profit
cp, expected_profit = gp.multidict({
    ('k1', 'p1'): 2000,
    ('k1', 'p2'): 1000,
    ('k2', 'p1'): 3000,
    ('k2', 'p2'): 2000
})


### Expected cost

The expected cost of offering a product to a customer in a cluster is shown in the following table.

| <i></i> | Product 1 | Product 2 |
| --- | --- |  --- |
| cluster 1 | $\$200$ | $\$100$ |
| cluster 2 | $\$300$ | $\$200$ |

In [3]:
# Expected cost

cp, expected_cost = gp.multidict({
    ('k1', 'p1'): 200,
    ('k1', 'p2'): 100,
    ('k2', 'p1'): 300,
    ('k2', 'p2'): 200
})

### Number of customers
The number of customers in each cluster can be seen in the following table.

| <i></i> | Num. Customers | 
| --- | --- |
| cluster 1 | 5 |
| cluster 2 | 5 | 

In [4]:
# Num of customers in each cluster

clusters, number_customers = gp.multidict({
    ('k1'): 5,
    ('k2'): 5
})

### Minimum number of offers

The minimum number of offers of each product is provided in the following table.

| <i></i> | Min Offers | 
| --- | --- |
| product 1 | 2 |
| product 2 | 2 | 

In [5]:
# Minimum number offers for each product

products, min_offers = gp.multidict({
    ('p1'): 2,
    ('p2'): 2
})

### Scalars

The corporate hurdle-rate is twenty percent ($R = 0.20$).

The budget available for the marketing campaign is $\$200$.


In [6]:
# Scalars

R = 0.20

#tight budget
budget = 200 

## Tactical Model Formulation
 

### Decision Variables
$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

In [7]:
# Declare and initialize model
mt = gp.Model('Tactical')

### Decisions variables

# Allocation of product offers to customers in clusters.

y = mt.addVars(cp, name="allocate")

# Budget correction

z = mt.addVar(name="budget_correction")

Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-


### Constraints

- **Number of offers**. Maximum number of offers of products for each cluster.

\begin{equation}
\sum_{j \in J} y_{k,j} \leq N_{k} \quad \forall k \in K
\tag{1}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$N_{k}$: Number of customers in cluster $k \in K$.

In [8]:
### Constraints

# Constraint on number of offers at each cluster

maxOffers_cons = mt.addConstrs((y.sum(k,'*') <= number_customers[k]  for k in clusters), name='maxOffers')


### Constraints

- **Budget**. The marketing campaign budget constraint enforces that the total cost of the campaign should be less than the budget campaign. There is the possibility of increasing the budget to ensure the feasibility of the model, the minimum number of offers for all the product may require this increase in the budget.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j} \leq B + z
\tag{2}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.

$B$: Marketing campaign budget.

In [9]:
# Budget constraint

budget_con = mt.addConstr((y.prod(expected_cost) - z <= budget), name='budget')


### Constraints

- **Offers limit**. Minimum number of offers of each product.

\begin{equation}
\sum_{k \in K} y_{k,j} \geq Q_{j}  \quad \forall j \in J
\tag{3}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$Q_{j}$: Minimum number of offers of product $j \in J$.

In [10]:
# Constraints on min number of offers of each product

minOffers_cons = mt.addConstrs( (y.sum('*',j) >= min_offers[j] for j in products), name='min_offers')


### Constraints


- **ROI**. The minimum ROI constraint ensures that the ratio of total profits over cost is at least  one plus the corporate hurdle rate.

\begin{equation}
\sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} \geq (1+R) \cdot \sum_{k \in K} \sum_{j \in J} \nu_{k,j} \cdot y_{k,j}
\tag{4}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$\pi_{k,j}$: Expected profit to the bank from the offer of product $j \in J$ to an average customer of cluster $k \in K$.

$\nu_{k,j}$: Average variable cost associated with the offer of product  $j \in J$ to an average customer of cluster $k \in K$.

$R$: Corporate hurdle rate.

In [11]:
# Constraint to ensure minimum ROI

ROI_con = mt.addConstr((y.prod(expected_profit) - (1 + R)*y.prod(expected_cost) >= 0), name='ROI')

### Objective Function
- **Total profit**. Maximize total expected profit from marketing campaign and heavily penalize any correction to the budget.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K} \sum_{j \in J} \pi_{k,j} \cdot y_{k,j} - M \cdot z
\tag{0}
\end{equation}

Where

$y_{k,j} \geq 0$: Number of customers in cluster $k \in K$ that are offered product $j \in J$.

$z \geq 0$: Increase in budget in order to have a feasible campaign.

$\pi_{k,j}$: Expected profit to the bank from the offer of product $j \in J$ to an average customer of cluster $k \in K$.

**Note:** The value of $M$ should be higher than any of the expected profits to ensure that the budget is increased only when the model is infeasible if this parameter is not increased.

In [12]:
### Objective function

# Maximize total expected profit

M = 10000

mt.setObjective(y.prod(expected_profit) -M*z, GRB.MAXIMIZE)

In [13]:
# Verify model formulation

mt.write('tactical.lp')

In [14]:
# Run optimization engine

mt.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 6 rows, 5 columns and 17 nonzeros
Model fingerprint: 0x1eac2f22
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+03, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+04   8.787500e+01   0.000000e+00      0s
       4   -3.9940000e+06   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective -3.994000000e+06


In [15]:
### Output Reports

# Optimal allocation of product offers to clusters

total_expected_profit = 0
total_expected_cost = 0

print("\nOptimal allocation of product offers to clusters.")
print("___________________________________________________")
for k,p in cp:
    if y[k,p].x > 1e-6:
        #print(y[k,p].varName, y[k,p].x)
        print(f"The number of customers in cluster {k} that gets an offer of product {p} is: {y[k,p].x}")
        total_expected_profit += expected_profit[k,p]*y[k,p].x
        total_expected_cost += expected_cost[k,p]*y[k,p].x

increased_budget = '${:,.2f}'.format(z.x)
print(f"\nThe increase correction in the campaign budget is {increased_budget}.")

# Financial reports

optimal_ROI = round(100*total_expected_profit/total_expected_cost,2)
min_ROI = round(100*(1+R),2)

money_expected_profit = '${:,.2f}'.format(total_expected_profit)
money_expected_cost = '${:,.2f}'.format(total_expected_cost)
money_budget = '${:,.2f}'.format(budget)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total expected profit is {money_expected_profit}.")
print(f"Optimal total expected cost is {money_expected_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {optimal_ROI}% with a minimum ROI of  {min_ROI}%.")


Optimal allocation of product offers to clusters.
___________________________________________________
The number of customers in cluster k1 that gets an offer of product p1 is: 2.0
The number of customers in cluster k1 that gets an offer of product p2 is: 2.0

The increase correction in the campaign budget is $400.00.

Financial reports.
___________________________________________________
Optimal total expected profit is $6,000.00.
Optimal total expected cost is $600.00 with a budget of $200.00 and an extra amount of $400.00.
Optimal ROI is 1000.0% with a minimum ROI of  120.0%.


## Analysis

The cost of allocating products to clusters required an increase in the budget of $\$400$. The total expected profit is 
$\$6,000$. The total expected cost is $\$600$ which is equal to the original budget of $\$200$ plus the increase of $\$400$. The expected ROI is 1,000 % which is much higher than the minimum ROI required.

## Operational Model Formulation

### Customer expected profit

In [16]:
### Sets

customers = ['c1', 'c2','c3','c4','c5','c6','c7','c8','c9','c10']

### Parameters

# Expected profit from a product offering for each customer in each cluster
ccp, customer_profit = gp.multidict({
    ('k1', 'c1', 'p1'): 2050,
    ('k1', 'c1', 'p2'): 1050,
    ('k1', 'c2', 'p1'): 1950,
    ('k1', 'c2', 'p2'): 950,
    ('k1', 'c3', 'p1'): 2000,
    ('k1', 'c3', 'p2'): 1000,
    ('k1', 'c4', 'p1'): 2100,
    ('k1', 'c4', 'p2'): 1100,
    ('k1', 'c5', 'p1'): 1900,
    ('k1', 'c5', 'p2'): 900,
    ('k2', 'c6', 'p1'): 3000,
    ('k2', 'c6', 'p2'): 2000,
    ('k2', 'c7', 'p1'): 2900,
    ('k2', 'c7', 'p2'): 1900,
    ('k2', 'c8', 'p1'): 3050,
    ('k2', 'c8','p2'): 2050,
    ('k2', 'c9', 'p1'): 3100,
    ('k2', 'c9', 'p2'): 3100,
    ('k2', 'c10', 'p1'): 2950,
    ('k2', 'c10', 'p2'): 2950   
})

### Customer offering cost

In [17]:
# Customer cost of offering a product at a cluster

ccp, customer_cost = gp.multidict({
    ('k1', 'c1', 'p1'): 205,
    ('k1', 'c1', 'p2'): 105,
    ('k1', 'c2', 'p1'): 195,
    ('k1', 'c2', 'p2'): 95,
    ('k1', 'c3', 'p1'): 200,
    ('k1', 'c3', 'p2'): 100,
    ('k1', 'c4', 'p1'): 210,
    ('k1', 'c4', 'p2'): 110,
    ('k1', 'c5', 'p1'): 190,
    ('k1', 'c5', 'p2'): 90,
    ('k2', 'c6', 'p1'): 300,
    ('k2', 'c6', 'p2'): 200,
    ('k2', 'c7', 'p1'): 290,
    ('k2', 'c7', 'p2'): 190,
    ('k2', 'c8', 'p1'): 305,
    ('k2', 'c8','p2'): 205,
    ('k2', 'c9', 'p1'): 310,
    ('k2', 'c9', 'p2'): 310,
    ('k2', 'c10', 'p1'): 295,
    ('k2', 'c10', 'p2'): 295   
})

## Operational Model Formulation


### Decision Variables
$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

In [18]:
# Declare and initialize model
mo = gp.Model('Operational')

### Decision variables

x = mo.addVars(ccp, vtype=GRB.BINARY, name="assign")

### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

$Y_{k,j} = \lfloor y_{k,j} \rfloor $: Number of customers in cluster k that will get an offer of product $j \in J^{k}$.




In [19]:
# Product offers constraint

productOffers = {}

for k in clusters:
    for j in products:
            productOffers[k,j] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffers_' + str(k) + ',' + str(j) )


### Constraints


- **Offers limit**. At most one product may be offered to a customer of a cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

In [20]:
# limit on the number of offers to each customer in a cluster.

ki = [('k1', 'c1'), 
      ('k1', 'c2'), 
      ('k1', 'c3'),
      ('k1', 'c4'), 
      ('k1', 'c5'), 
      ('k2', 'c6'), 
      ('k2', 'c7'), 
      ('k2', 'c8'), 
      ('k2', 'c9'), 
      ('k2', 'c10')]

customerOffers = {}

for k,i in ki:
    customerOffers[k,i] = mo.addConstr(gp.quicksum(x[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffers_' + str(k) + ',' + str(i) )

### Objective Function

- **Total profit**. Maximize total individual expected profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

Where

$x_{k,i,j} \in \{0,1 \}$: This variable is equal to 1, if product $j \in J^{k}$  is offered to customer $i \in I^{k}$, and 0 otherwise.

$r_{k,i,j}$: Expected individual profit of customer $i \in I^{k}$ from  offer of product $j \in J^{k}$. 



In [21]:
### Objective function

# Maximoze total profit

mo.setObjective(x.prod(customer_profit), GRB.MAXIMIZE)

In [22]:
# Verify model formulation

mo.write('operational.lp')

# Run optimization engine

mo.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 14 rows, 20 columns and 40 nonzeros
Model fingerprint: 0xe1e9d99f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6050.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 20 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.100000e+03, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6100.0000000 6100.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 6100 6050 

Optimal 

In [23]:
### Output Reports

# Optimal assignment of product offers to customers

total_customer_profit = 0
total_customer_cost = 0

kvalue = None
first = True
num_assignments = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalue:
        prevk = kvalue
        kvalue = k
        if not first:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevk} is {num_assignments}")
            print("___________________________________________________")
            num_assignments = 0
        if first:
            first = False
    if x[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profit = '${:,.2f}'.format(customer_profit[k,i,j])
        cost = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profit} at a cost of {cost}")
        total_customer_profit += customer_profit[k,i,j]*x[k,i,j].x
        total_customer_cost += customer_cost[k,i,j]*x[k,i,j].x
        num_assignments += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalue} is {num_assignments}")
print("___________________________________________________\n")
        
# Financial reports

customers_ROI = round(100*total_customer_profit/total_customer_cost,2)

money_customers_profit = '${:,.2f}'.format(total_customer_profit)
money_customers_cost = '${:,.2f}'.format(total_customer_cost)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profit}.")
print(f"Optimal total customers cost is {money_customers_cost} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROI}% with a minimum ROI of  {min_ROI}%.")
        



Optimal assignment of product offers to customers.
___________________________________________________
Customer c1 in cluster k1 gets an offer of product p2:
The expected profit is $1,050.00 at a cost of $105.00
Customer c2 in cluster k1 gets an offer of product p2:
The expected profit is $950.00 at a cost of $95.00
Customer c3 in cluster k1 gets an offer of product p1:
The expected profit is $2,000.00 at a cost of $200.00
Customer c4 in cluster k1 gets an offer of product p1:
The expected profit is $2,100.00 at a cost of $210.00
___________________________________________________
Number of assignments in cluster k1 is 4
___________________________________________________
___________________________________________________
Number of assignments in cluster k2 is 0
___________________________________________________


Financial reports.
___________________________________________________
Optimal total customers profit is $6,100.00.
Optimal total customers cost is $610.00 with a budget o

## Analysis
Each customer got, at most, one product offer.  Product p2 is offered to customers c1 and c2, and product p1 is offered to customers c3 and c4. Products p1 and p2 are offerred to at least two customers -this is a constraint from the tactical model. Observe that to ensure these hard business constraints, the budget needs to be increased by $\$400$

The cost of assigning  products to customers is $\$610$, which slightly violates the total budget available of $\$600$. The total customers profit is $\$6,100$. The  ROI is 1,000 %, which is much higher than the minimum ROI required.

If the total available budget needs to be enforced, the following constraint can be added to the operational model:

- **Budget**. Enforce budget constraint.

\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{4}
\end{equation}

The new budget is the original budget plus the correction, that is  $B' = B + z$

## Scenario 1
Enforce total budget available constraint. In this case, the operational model is:

### Objective function

- **Total profit**. Maximize total individual expected profit.

\begin{equation}
\text{Max} \quad Z = \sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} r_{k,i,j} \cdot x_{k,i,j}
\tag{0}
\end{equation}

### Constraints

- **Product offers**. Allocate offers of a product to customers of each cluster.

\begin{equation}
\sum_{i \in  I^{k}}  x_{k,i,j} = Y_{k,j}  \quad \forall j \in J^{k}, k \in K
\tag{1}
\end{equation}

- **Offers limit**. At most one product may be offered to a customer in each cluster.

\begin{equation}
\sum_{j \in J^{k}} x_{k,i,j} \leq 1 \quad \forall i \in I^{k}, k \in K
\tag{2}
\end{equation}

- **Budget**. Enforce budget constraint.

\begin{equation}
\sum_{k \in K}  \sum_{i \in I^{k}} \sum_{j \in J^{k}} c_{k,i,j} \cdot x_{k,i,j} \leq B'
\tag{3}
\end{equation}

In [24]:
### Operational model enforcing constraint for total budget available 

# Declare and initialize model
mob = gp.Model('OperationalB')

### Decision variables

xb = mob.addVars(ccp, vtype=GRB.BINARY, name="assign")

In [25]:
# Product offers constraint

productOffersb = {}

for k in clusters:
    for j in products:
            productOffersb[k,j] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,i,jj in ccp if (kk ==k and jj == j)) == 
                                              int(y[k,j].x), name='prodOffersb_' + str(k) + ',' + str(j) )

In [26]:
# limit on the number of offers to each customer in a cluster.

customerOffersb = {}

for k,i in ki:
    customerOffersb[k,i] = mob.addConstr(gp.quicksum(xb[k,i,j] for kk,ii,j in ccp if (kk == k and ii == i) ) <= 1, 
                                          name ='custOffersb_' + str(k) + ',' + str(i) )

In [27]:
# budget constraint

# New budget
new_budget = budget + z.x

totBudget = mob.addConstr(xb.prod(customer_cost) <= new_budget, name='total_budget')

In [28]:
### Objective function

# Maximize total profit

mob.setObjective(xb.prod(customer_profit), GRB.MAXIMIZE)

In [29]:
# Verify model formulation

mob.write('operationalB.lp')

# Run optimization engine

mob.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 15 rows, 20 columns and 60 nonzeros
Model fingerprint: 0x96da858f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 5950.0000000
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 28 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 6.000000e+03, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6000.0000000 6000.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 6000 5950 

Optimal 

In [30]:
### Output Reports

# Optimal assignment of product offers to customers

total_customer_profitb = 0
total_customer_costb = 0

kvalueb = None
firstb = True
num_assignmentsb = 0

print("\nOptimal assignment of product offers to customers.")
print("___________________________________________________")
for k,i,j in ccp:
    if k != kvalueb:
        prevkb = kvalueb
        kvalueb = k
        if not firstb:
            print("___________________________________________________")
            print(f"Number of assignments in cluster {prevkb} is {num_assignmentsb}")
            print("___________________________________________________")
            num_assignmentsb = 0
        if firstb:
            firstb = False
    if xb[k,i,j].x > 0.5:
        #print(x[k,i,j].varName, x[k,i,j].x)
        profitb = '${:,.2f}'.format(customer_profit[k,i,j])
        costb = '${:,.2f}'.format(customer_cost[k,i,j])
        print(f"Customer {i} in cluster {k} gets an offer of product {j}:")
        print(f"The expected profit is {profitb} at a cost of {costb}")
        total_customer_profitb += customer_profit[k,i,j]*xb[k,i,j].x
        total_customer_costb += customer_cost[k,i,j]*xb[k,i,j].x
        num_assignmentsb += 1
print("___________________________________________________")
print(f"Number of assignments in cluster {kvalueb} is {num_assignmentsb}")
print("___________________________________________________\n")
        
# Financial reports

customers_ROIb = round(100*total_customer_profitb/total_customer_costb,2)

money_customers_profitb = '${:,.2f}'.format(total_customer_profitb)
money_customers_costb = '${:,.2f}'.format(total_customer_costb)

print(f"\nFinancial reports.")
print("___________________________________________________")
print(f"Optimal total customers profit is {money_customers_profitb}.")
print(f"Optimal total customers cost is {money_customers_costb} with a budget of {money_budget} and an extra amount of {increased_budget}.")
print(f"Optimal ROI is {customers_ROIb}% with a minimum ROI of  {min_ROI}%.")


Optimal assignment of product offers to customers.
___________________________________________________
Customer c1 in cluster k1 gets an offer of product p2:
The expected profit is $1,050.00 at a cost of $105.00
Customer c2 in cluster k1 gets an offer of product p1:
The expected profit is $1,950.00 at a cost of $195.00
Customer c4 in cluster k1 gets an offer of product p1:
The expected profit is $2,100.00 at a cost of $210.00
Customer c5 in cluster k1 gets an offer of product p2:
The expected profit is $900.00 at a cost of $90.00
___________________________________________________
Number of assignments in cluster k1 is 4
___________________________________________________
___________________________________________________
Number of assignments in cluster k2 is 0
___________________________________________________


Financial reports.
___________________________________________________
Optimal total customers profit is $6,000.00.
Optimal total customers cost is $600.00 with a budget o

## Analysis
Each customer got, at most, one product offer. Products p1 and p2 are offerred to at least two customers. The cost of assigning  products to customers is $\$600$, which is equal to the total budget available. The total customers profit is $\$6,000$. The  ROI is 1,000 %, which is much higher than the minimum ROI required.

In this scenario, we enforce the total available budget constraint and we get a different distribution for the assignment. 
Product p1 is offered to customers c2 and c4, and product p2 is offered to customers c1 and c5. 

##  Conclusion

In this Jupyter Notebook, we discussed the importance of marketing campaigns for the banking industry. We discussed that machine learning predictive response models can be used to provide the input data of a marketing campaign optimization problem. We showed how the marketing campaign optimization problem can be decomposed into a tactical problem and an operational problem.

The tactical problem is formulated as a linear programming problem where we aggregate data generated by the machine learning predictive response models.

The solution of the tactical problem determines what products to offer to customers in clusters while maximizing the marketing campaign expected profits and considering the following constraints: 

* limits on funding available for the campaign.
* restrictions on the minimum number of product offers that can be made in a campaign.
* campaign return-on-investment hurdle rates that must be met.

The operational problem is formulated as a MIP model, where the expected customer  profits and the output of the tactical model can be used as inputs to assign products offers to individual customers in such way that the total customers profit is maximized. We considered two cases for the operational problem. In the first case, the total available budget determined by tactical problem is not enforced. This means that the optimal solution of this problem might have slight violations of the total available budget. In the second case, we enforce a total available budget.


##  References

[1] M. D. Cohen. *Exploiting response models—optimizing cross-sell and up-sell opportunities in banking.* Information Systems. Vol. 29. issue 4, June 2004, Pages 327-341

Copyright © 2020 Gurobi Optimization, LLC